In [5]:
import joblib


import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
df = joblib.load('data/txtclean2')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76640 entries, 0 to 76639
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   app_id               76640 non-null  int64         
 1   dlc                  76640 non-null  int64         
 2   recommendationid     76640 non-null  object        
 3   review               76640 non-null  object        
 4   voted_up             76640 non-null  object        
 5   votes_up             76640 non-null  float64       
 6   votes_funny          76640 non-null  float64       
 7   weighted_vote_score  76640 non-null  float64       
 8   review_created       76640 non-null  datetime64[ns]
 9   review_updated       76640 non-null  datetime64[ns]
 10  review_year          76640 non-null  int64         
 11  steamid              76640 non-null  object        
 12  num_games_owned      76640 non-null  int64         
 13  num_reviews          76640 non-

In [9]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['TJBs', 'YET', 'V10', 'EMPRAH', 'The', 'it', 'td', 'This', 'td', 'It'])

In [10]:

# Convert to list
data = df.review.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['I got this back in 2010 and it was around $2, but now its $8? For four '
 'skins? Dont get me wrong, the skins are quite good and all, but it is a '
 'complete ripoff at this price.']


In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['got', 'this', 'back', 'in', 'and', 'it', 'was', 'around', 'but', 'now', 'its', 'for', 'four', 'skins', 'dont', 'get', 'me', 'wrong', 'the', 'skins', 'are', 'quite', 'good', 'and', 'all', 'but', 'it', 'is', 'complete', 'ripoff', 'at', 'this', 'price']]


In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['got', 'this', 'back', 'in', 'and', 'it', 'was', 'around', 'but', 'now', 'its', 'for', 'four', 'skins', 'dont', 'get', 'me', 'wrong', 'the', 'skins', 'are', 'quite', 'good', 'and', 'all', 'but', 'it', 'is', 'complete', 'ripoff', 'at', 'this', 'price']


In [13]:
#Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [19]:
#call functions in order
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['get', 'back', 'around', 'skin', 'do', 'get', 'wrong', 'skin', 'quite', 'good', 'complete', 'ripoff', 'price']]


In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1)]]


In [21]:
id2word[0]

'around'

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('around', 1),
  ('back', 1),
  ('complete', 1),
  ('do', 1),
  ('get', 2),
  ('good', 1),
  ('price', 1),
  ('quite', 1),
  ('ripoff', 1),
  ('skin', 2),
  ('wrong', 1)]]

In [23]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [25]:
#joblib.dump(lda_model, 'data/lda_model1')

['data/lda_model1']

In [24]:
# Print the Keyword in the 8 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.050*"game" + 0.029*"get" + 0.026*"dlc" + 0.024*"good" + 0.023*"do" + '
  '0.017*"buy" + 0.016*"play" + 0.016*"really" + 0.016*"well" + 0.013*"great"'),
 (1,
  '0.046*"pay" + 0.042*"weapon" + 0.036*"overall" + 0.031*"wait" + '
  '0.022*"design" + 0.017*"miss" + 0.015*"hit" + 0.015*"gun" + 0.014*"super" + '
  '0.013*"community"'),
 (2,
  '0.017*"music" + 0.016*"new" + 0.015*"ship" + 0.014*"addition" + '
  '0.014*"pass" + 0.012*"yet" + 0.011*"table" + 0.010*"turn" + 0.010*"spoiler" '
  '+ 0.010*"build"'),
 (3,
  '0.047*"work" + 0.033*"ve" + 0.029*"support" + 0.021*"steam" + '
  '0.019*"scenario" + 0.018*"show" + 0.018*"keep" + 0.017*"never" + '
  '0.015*"devs" + 0.015*"developer"'),
 (4,
  '0.041*"make" + 0.023*"see" + 0.022*"feel" + 0.018*"lot" + 0.017*"character" '
  '+ 0.016*"pretty" + 0.015*"re" + 0.014*"be" + 0.013*"story" + '
  '0.012*"review"'),
 (5,
  '0.073*"mode" + 0.064*"amazing" + 0.041*"less" + 0.034*"kind" + '
  '0.023*"require" + 0.020*"understand" + 0.020*"dollar

In [26]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.336379  0.115383       1        1  47.044907
4      0.144598 -0.405555       2        1  18.200598
7      0.160203  0.174948       3        1  10.292567
2     -0.030184  0.110243       4        1   9.215689
3     -0.050535 -0.027153       5        1   8.018934
1     -0.172997 -0.002963       6        1   4.536812
5     -0.201359  0.014579       7        1   2.050784
6     -0.186105  0.020519       8        1   0.639710, topic_info=           Term          Freq         Total Category  logprob  loglift
29         game  45662.000000  45662.000000  Default  30.0000  30.0000
176        make  14306.000000  14306.000000  Default  29.0000  29.0000
4           get  25967.000000  25967.000000  Default  28.0000  28.0000
519        work   7233.000000   7233.000000  Default  27.0000  27.0000
78          dlc  23651.000000  23651.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
5678       wolf     58.595141     59.521018   Topic8  -5.3499   5.0362
965         usa     55.029040     55.954656   Topic8  -5.4127   5.0352
2570      realm     48.312825     49.238451   Topic8  -5.5428   5.0329
247   everytime     46.061605     46.986741   Topic8  -5.5905   5.0320
7343     refuse     43.282915     44.207692   Topic8  -5.6528   5.0308

[297 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
839        8  0.998561  absolute
13         1  0.999944       add
968        4  0.999723  addition
245        8  0.994252       ago
8218       3  0.999398  aircraft
...      ...       ...       ...
24609      7  0.914046  wrestler
1336       7  0.998420       wtf
695        5  0.999590      year
617        4  0.999580       yet
10895      7  0.996739       zen

[294 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 8, 3, 4, 2, 6, 7])

In [31]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [29]:
topics_df1 = pd.DataFrame()
topics_df2 = pd.DataFrame()
topics_df3 = pd.DataFrame()

for i, row_list in enumerate(lda_model[corpus]):
    row = row_list[0] if lda_model.per_word_topics else row_list            
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if len(row) >= 3:        
            if j ==0:
                topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            elif j ==1:
                topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            elif j ==2:
                topics_df3 = topics_df3.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            else:
                break
        elif len(row) == 2:
            if j ==0:
                topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
            elif j ==1:
                topics_df2 = topics_df2.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
                topics_df3 = topics_df3.append(pd.Series(['-', '-']), ignore_index=True)
        elif len(row) == 1:
                topics_df1 = topics_df1.append(pd.Series([int(topic_num), prop_topic]), ignore_index=True)
                topics_df2 = topics_df2.append(pd.Series(['-', '-']), ignore_index=True)  
                topics_df3 = topics_df3.append(pd.Series(['-', '-']), ignore_index=True)        
                
            
topics_df1.rename(columns={0:'1st Topic', 1:'1st Topic Contribution'}, inplace=True)
topics_df2.rename(columns={0:'2nd Topic', 1:'2nd Topic Contribution'}, inplace=True)
topics_df3.rename(columns={0:'3rd Topic', 1:'3rd Topic Contribution'}, inplace=True)

topics_comb = pd.concat([topics_df1, topics_df2, topics_df3],  axis=1, sort=False)

#Join topics dataframe to original data
#new_df = pd.concat([data_ready, topics_comb], axis=1, sort=False)


NameError: name 'data_ready' is not defined

In [30]:
topics_comb

,1st Topic,1st Topic Contribution,2nd Topic,2nd Topic Contribution,3rd Topic,3rd Topic Contribution
0,0.0,0.562136,4.0,0.190122,7.0,0.059228
1,0.0,0.520859,4.0,0.165025,7.0,0.081263
2,4.0,0.503444,0.0,0.386642,1.0,0.032853
3,0.0,0.447371,7.0,0.230889,4.0,0.137509
4,0.0,0.513402,4.0,0.206071,7.0,0.076301
...,...,...,...,...,...,...
76635,0.0,0.482119,4.0,0.205010,7.0,0.090810
76636,0.0,0.486952,4.0,0.154192,7.0,0.108610
76637,0.0,0.503398,4.0,0.193177,7.0,0.078505
76638,0.0,0.519616,4.0,0.154194,3.0,0.108297


In [ ]:
#Join topics dataframe to original data
new_df = pd.concat([data, topics_comb], axis=1, sort=False)



In [35]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = row_list[0] if lda_model.per_word_topics else row_list  
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...","I got this back in 2010 and it was around $2, ..."
1,1,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",DJ Scully nuff said.
2,2,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",These Killing Floor characters really adds to ...
3,3,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",Guy in a HaZmat suit... cool Some guy with som...
4,4,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",This DLC has the best looking character models...
5,5,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",As far as [url=]Killing Floor[/url]s DLC chara...
6,6,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",DOSH! Spend it while you can! Twelve cents per...
7,7,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...","I bought this pack first because of DJ Scully,..."
8,8,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",my balls itch holy shit
9,9,0.0,0.4977,"game, get, dlc, good, do, buy, play, really, w...",Remember when these character DLCs only cost t...
